<a href="https://colab.research.google.com/github/ronykroy/DNN-NLP-and-other-stuff/blob/master/Elmo_NewsGroups_MultiClassClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# hat tip Jason zwieg... @ stong IO
# https://github.com/strongio/keras-elmo/blob/master/Elmo%20Keras.ipynb

In [0]:
# Import our dependencies
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer
import numpy as np

# Initialize session
sess = tf.Session()
K.set_session(sess)

In [0]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import numpy as np
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
df = pd.DataFrame({'label':dataset.target, 'text':dataset.data})

In [0]:
#df["Train"] = np.random.rand(len(df)) < 0.8 # get a col full of train boolean 
#df.Train.value_counts()

In [0]:
from sklearn.model_selection import train_test_split
df_trn, df_test = train_test_split(df, stratify = df['label'], test_size = 0.15, random_state = 11) # test_size is debatable

In [18]:
df_trn.head(3)

,label,text
1401,0,In <lsran6INN14a@exodus.Eng.Sun.COM> emarsh@he...
9452,6,"Nintendo 8 bit system, power pad, light gun (z..."
3549,16,}In article <C5D4Hv.8Dp@undergrad.math.uwaterl...


In [0]:
# Create a custom layer that allows us to update weights (lambda layers do not have trainable parameters!)

class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape): # TF hub ki Jai....
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        #self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        self.trainable_weights += tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [0]:
# Function to build model
def build_model(): 
  input_text = layers.Input(shape=(1,), dtype="string")
  embedding = ElmoEmbeddingLayer()(input_text)
  dense = layers.Dense(256, activation='relu')(embedding)
  pred = layers.Dense(20, activation='sigmoid')(dense) # because 20 categories....

  model = Model(inputs=[input_text], outputs=pred)

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # was binary cross entropy
  # Binary cross-entropy is for multi-label classifications 
  # categorical cross entropy is for multi-class classification where each example belongs to a single class.
  #https://datascience.stackexchange.com/questions/41921
  #https://stats.stackexchange.com/questions/357541/
  model.summary()
  
  return model

In [0]:
from keras.utils import to_categorical
# toa ddress the error message
# when all categories were in one col

In [23]:
# Create datasets (Only take up to 150 words for memory)
train_text = df_trn['text'].tolist()
train_text = [' '.join(t.split()[0:150]) for t in train_text]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = df_trn['label'].tolist() # polarity is what we are trining on... 
train_label = to_categorical(train_label)

test_text = df_test['text'].tolist()
test_text = [' '.join(t.split()[0:150]) for t in test_text]
test_text = np.array(test_text, dtype=object)[:, np.newaxis]
test_label = df_test['label'].tolist() # polarity is what we are trining on... ditto for the test set
test_label = to_categorical(test_label)


"""
  You are passing a target array of shape (9063, 1) while using as loss `categorical_crossentropy`. 
  `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). 
  If your targets are integer classes, you can convert them to the expected format via:
```
from keras.utils import to_categorical
y_binary = to_categorical(y_int)
  """

'\n  You are passing a target array of shape (9063, 1) while using as loss `categorical_crossentropy`. \n  `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). \n  If your targets are integer classes, you can convert them to the expected format via:\n```\nfrom keras.utils import to_categorical\ny_binary = to_categorical(y_int)\n  '

In [0]:
# Build and fit
model = build_model()
model.fit(train_text, 
          train_label,
          validation_data=(test_text, test_label),
          epochs=3,
          batch_size=32)

Categorical ross entropy    
1 epoch: 52% accuracy  
3 Epochs :  56.95%
TODO:   
same with sparse categorical cross entropy  
3 epochs:   